# Loading and Cleaning the Data

## Step 1: Importing Libraries

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt 
import pickle

## Step 2: Reading in the Turnstile Data

In [11]:
# Define the weeks to retrieve from the MTA database
def list_weeks(list_of_dates):
    weeks = []
    for date in list_of_dates:
        week_string = date.strftime('%y%m%d')
        weeks.append(week_string)
    return weeks        

# We want to pull in data from a week in mid-January and mid-May of 2018 and 2019
dates = [dt.datetime(2018,1,13),dt.datetime(2018,4,14),dt.datetime(2019,1,12),dt.datetime(2019,4,13)]
weeks = list_weeks(dates)
weeks

['180113', '180414', '190112', '190413']

In [12]:
# Read in the data for the desired weeks
def readTurnstileData(week_string):
    """
    This function reads in data from an online MTA Turnstile dataset into a DataFrame
    ---
    input: link to dataset
    output: DataFrame
    """
    cols = ['control_area','unit','scp','station','line_name','division','date','time',
        'desc','entries','exits']
    
    link = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_' + week_string + '.txt'
    turnstile_data = pd.read_csv(link, header = 0, names = cols)
    return turnstile_data

# Create a single DataFrame containing all weeks
def createTurnstileDataFrame(list_of_weeks):
    df = pd.DataFrame()
    df_chunk_list = []
    for date in list_of_weeks:
        df_chunk = readTurnstileData(date)
        df_chunk_list.append(df_chunk)
    df = pd.concat(df_chunk_list)
    return df

# Clean up date/time info
def formatDateTime(df):
    """
    This function converts the date and time into DateTime format in a single column
    and deletes the unformatted date and time columns
    
    Note: only run once per DataFrame, will result in error otherwise
    ---
    input: DataFrame
    output: DataFrame
    """
    
    #convert date and time to DateTime format in a single column
    df['concat_date_time'] = df['date'] + ' ' + df['time']
    df['date_time'] = pd.to_datetime(df.concat_date_time,format = '%m/%d/%Y %H:%M:%S')
    
    #delete unformatted date and time columns
    del df['concat_date_time']
    del df['date']
    del df['time']
    return df

def read_and_format_turnstile_data(list_of_weeks):
    """
    This function reads in Turnstile data from online and returns a DataFrame with 
    with the date and time information converted to a single DateTime column
    ---
    input: link
    output: DataFrame
    """
    df = createTurnstileDataFrame(list_of_weeks)
    df = formatDateTime(df)
    return df

In [13]:
# Read in the data from the specified weeks
orig_df = read_and_format_turnstile_data(weeks)

In [14]:
# Save raw data to access easily later
with open('data/orig_df.pickle', 'wb') as to_write:
    pickle.dump(orig_df, to_write)
del orig_df

## Step 2ish: Read in Station Location Data

## Step 2.5: read in data from pickle

In [27]:
# Read in the data from the pickle file if you are starting here
# If you started from the beginning of the notebook, comment out the code below

with open('data/orig_df.pickle','rb') as read_file:
    orig_df = pickle.load(read_file)
df = orig_df

## Step 3: Organizing the Data

In [28]:
# Reset indexing on the DataFrame
df.reset_index(inplace=True)

# Delete old indexes column
# Only run once
del df['index']

In [29]:
# Create a column for weekday
df['weekday'] = df[['date_time']].apply(lambda x: x['date_time'].dayofweek,axis=1)

In [30]:
# Create a column for year
df['year'] = df[['date_time']].apply(lambda x: x['date_time'].year,axis=1)

In [31]:
# Create a column for the time of day
df['hour'] = df[['date_time']].apply(lambda x: x['date_time'].hour,axis=1)

In [32]:
# Create a column with unique ID for each turnstile
df['turnstile_id'] = df.groupby(['control_area','unit','scp','station','year']).ngroup()

In [ ]:
# Create columns for latitude/longitude and borough of each turnstile

## Step 3: Cleaning the Data

In [33]:
# Sort DataFrame by turnstile and date
df.sort_values(['turnstile_id','date_time'],inplace=True)
df.reset_index(drop=True,inplace=True)

# Find entries and exits differences per turnstile
df['exit_counts'] = abs(df.groupby('turnstile_id').exits.diff())
df['entry_counts'] = abs(df.groupby('turnstile_id').entries.diff())
    

In [34]:
df.describe()

,entries,exits,weekday,year,hour,turnstile_id,exit_counts,entry_counts
count,8.035100e+05,8.035100e+05,803510.000000,803510.000000,803510.000000,803510.000000,7.938810e+05,7.938810e+05
mean,4.005398e+07,3.301926e+07,2.990372,2018.505035,11.113836,4815.938751,3.928604e+04,4.588957e+04
std,2.076849e+08,1.924742e+08,1.995281,0.499975,6.919003,2785.148550,6.777609e+06,7.113515e+06
min,0.000000e+00,0.000000e+00,0.000000,2018.000000,0.000000,0.000000,0.000000e+00,0.000000e+00
25%,4.587942e+05,2.128798e+05,1.000000,2018.000000,5.000000,2409.000000,8.000000e+00,1.000000e+01
50%,2.454182e+06,1.389020e+06,3.000000,2019.000000,11.000000,4799.000000,5.100000e+01,7.400000e+01
75%,6.860751e+06,4.788484e+06,5.000000,2019.000000,17.000000,7231.000000,1.670000e+02,2.450000e+02
max,2.130144e+09,2.124178e+09,6.000000,2019.000000,23.000000,9628.000000,2.107450e+09,2.088571e+09


Max values on exit and entry counts are way too high

In [35]:
def findOutliers(df,column):
    """
    Returns outliers above the max limit for a column in a dataframe
    Adjust outlier cutoff to q75 + 4*iqr to include more data
    ---
    input: DataFrame, column
    output: DataFrame
    """
    q25,q50,q75 = df[column].quantile(q=[0.25,0.5,0.75])
    iqr = q75-q25
    #max limits to be considered an outlier
    max = q75 + 4*iqr
    #identify the points
    outlier_mask = [True if x > max else False for x in df[column]]
    print('{} outliers found out of {} data points, {}% of the data'.format(sum(outlier_mask),len(df[column]),100*(sum(outlier_mask)/len(df[column]))))
    return outlier_mask

In [43]:
#Get outliers for entries
print('Entry: ')
df['entry_outliers'] = findOutliers(df,'entry_counts')

#Get outliers for exits
print('\nExit:')
df['exit_outliers'] = findOutliers(df,'exit_counts')

#DataFrame with entry outliers removed
clean_df_entries = df.loc[~df['entry_outliers']]
print('\n{} points on entry side left after removing entry_counts outlier points'.format(clean_df_entries.shape[0]))

#DataFrame with exit outliers removed
clean_df_exits = df.loc[~df['exit_outliers']]
print('{} points on exit side left left after removing exit_counts outlier points'.format(clean_df_exits.shape[0]))


Entry: 
17089 outliers found out of 803510 data points, 2.1267936926733952% of the data

Exit:
26722 outliers found out of 803510 data points, 3.325658672574081% of the data

786421 points on entry side left after removing entry_counts outlier points
776788 points on exit side left left after removing exit_counts outlier points


In [44]:
# Eliminate Null values
# Delete rows with null values for entry_counts
clean_df_entries = clean_df_entries[~clean_df_entries.entry_counts.isnull()]
print('{} points left after removing entry_counts NaN values'.format(clean_df_entries.shape[0]))

# Delete rows with null values for exit_counts
clean_df_entries = clean_df_entries[~clean_df_entries.exit_counts.isnull()]
print('{} points left after removing entry_counts NaN values'.format(clean_df_exits.shape[0]))


776792 points left after removing entry_counts NaN values
776788 points left after removing entry_counts NaN values


In [47]:
# Merge the two DataFrames
clean_df = clean_df_entries.merge(clean_df_exits,left_on=list(clean_df_entries.columns), \
                                  right_on=list(clean_df_exits.columns),how='inner')
#Add a column for total traffic at each turnstile
clean_df['total_traffic'] = clean_df['entry_counts'] + clean_df['exit_counts']

In [48]:
print('{} rows left after cleaning the data, {}% of the original'.format(clean_df.shape[0],100*(clean_df.shape[0]/df.shape[0])))

758874 rows left after cleaning the data, 94.44487311918955% of the original


In [49]:
# Delete outlier ID columns
# Only run this cell once
del clean_df['entry_outliers']
del clean_df['exit_outliers']

In [50]:
# Save cleaned data to access easily later
with open('data/clean_df.pickle', 'wb') as to_write:
    pickle.dump(clean_df, to_write)

del clean_df

## Step 4.5: Read in the cleaned data

In [52]:
# Read in the data from the pickle file if you are starting here
# If you started from the beginning of the notebook, comment out the code below

import pickle

with open('data/clean_df.pickle','rb') as read_file:
    clean_df = pickle.load(read_file)

In [53]:
clean_df

,control_area,unit,scp,station,line_name,division,desc,entries,exits,date_time,weekday,year,hour,turnstile_id,exit_counts,entry_counts,total_traffic
0,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470223,2190140,2018-01-06 07:00:00,5,2018,7,0,17.0,7.0,24.0
1,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470256,2190229,2018-01-06 11:00:00,5,2018,11,0,89.0,33.0,122.0
2,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470379,2190299,2018-01-06 15:00:00,5,2018,15,0,70.0,123.0,193.0
3,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470665,2190366,2018-01-06 19:00:00,5,2018,19,0,67.0,286.0,353.0
4,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470809,2190398,2018-01-06 23:00:00,5,2018,23,0,32.0,144.0,176.0
5,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470843,2190405,2018-01-07 03:00:00,6,2018,3,0,7.0,34.0,41.0
6,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470849,2190411,2018-01-07 07:00:00,6,2018,7,0,6.0,6.0,12.0
7,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6470884,2190488,2018-01-07 11:00:00,6,2018,11,0,77.0,35.0,112.0
8,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6471042,2190530,2018-01-07 15:00:00,6,2018,15,0,42.0,158.0,200.0
9,A002,R051,02-00-00,59 ST,NQR456W,BMT,REGULAR,6471222,2190557,2018-01-07 19:00:00,6,2018,19,0,27.0,180.0,207.0


In [5]:
stations = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')
stations.nunique()

Station ID               493
Complex ID               445
GTFS Stop ID             496
Division                   4
Line                      34
Stop Name                377
Borough                    5
Daytime Routes            62
Structure                  6
GTFS Latitude            493
GTFS Longitude           493
North Direction Label     38
South Direction Label     34
dtype: int64

In [8]:
clean_df.nunique()

index            204594
control_area        749
unit                469
scp                 220
station             378
line_name           114
division              6
desc                  2
entries          603025
exits            586320
date_time         48715
weekday               7
year                  2
hour                 24
turnstile_id       9627
exit_counts         645
entry_counts       1186
total_traffic      1744
dtype: int64

In [70]:
station_stations = stations['Stop Name'].apply(str.lower).sort_values().reset_index()
turnstile_stations = pd.DataFrame(clean_df['station'].apply(str.lower).sort_values().unique()).reset_index()

In [71]:
print(station_stations.head(10))
print(turnstile_stations.head(10))

   index              Stop Name
0    118                   1 av
1    395                 103 st
2    309                 103 st
3    155                 103 st
4    450  103 st - corona plaza
5    193                 104 st
6     81                 104 st
7    394                 110 st
8    449                 111 st
9    194                 111 st
   index                0
0      0             1 av
1      1           103 st
2      2    103 st-corona
3      3           104 st
4      4           110 st
5      5           111 st
6      6           116 st
7      7  116 st-columbia
8      8           121 st
9      9           125 st


## Analyzing the Data

- Which station has the most foot traffic:
    - On weekdays in particular?
    - On which days?
    - At what times?


- Which stations are near college campuses?
- Which stations are in tech-heavy areas?
- Which stations are in wealthy neighborhoods?

In [ ]:
station_groupby = 

In [51]:
station_groupby = turnstile_data.groupby('station')

In [54]:
station_entries = station_groupby.entries.sum().sort_values(ascending = True)
station_entries

station
NEWARK HM HE            1073019
PATH WTC 2              7402804
NEWARK HW BMEBE        16359580
9TH STREET             18546632
ORCHARD BEACH          20787564
                       ...     
DEKALB AV          226731273077
23 ST              237675376663
TIMES SQ-42 ST     244851205878
125 ST             282278333960
42 ST-PORT AUTH    315905669087
Name: entries, Length: 378, dtype: int64